# Scrapeando letras de tango

Esto constituye el primer intento para generar un scrapper que pueda bajar todas las letras de tango de [todotango](http://www.todotango.com/). Para eso vamos a usar la librería `BeatifulSoup` que sirve para hacer webscraping.

**Nota:** Se agradece la colaboración de E. Piccolli en este notebook.

In [ ]:
import urllib2
from bs4 import BeautifulSoup as soup
from numpy import random
import multiprocessing as mp
import pandas as pd
import json, codecs
#%load_ext line_profiler

In [ ]:
index = urllib2.urlopen('http://www.todotango.com/musica/obras/all/-/0/0/').read()
index_soup = soup(index,"html.parser")

index_link = []
for link in index_soup.find_all('a'):
    index_link.append(link.get('href'))

index_link = index_link[72:(len(index_link)-15)]

In [ ]:
def scrap_tango(lista, save_file):
    canciones = []
    count = 0
    data = pd.DataFrame()
    for i in lista:
        # Open site and parse it

        r = urllib2.urlopen(i).read()
        page_soup = soup(r,"html.parser") 

        # get text from interesting sections            
        titulo = [span.string for span in page_soup.find_all('span', attrs={'id':'main_Tema1_lbl_Titulo'}) ]
        ritmo =  [span.string for  span in page_soup.find_all('span', attrs={'id':'main_Tema1_lbl_Ritmo'})]
        ano = [span.string for span in page_soup.find_all('span', attrs={'id':'main_Tema1_lbl_Ano'})]
        musica = [link.string  for link in page_soup.find_all('a', attrs={'id':'main_Tema1_Tema_Autores1_RP_TemasCreadores_AutoresMusica_hl_Creador_0'})]
        compositor = [link.string for link in page_soup.find_all('a', attrs={'id':'main_Tema1_Tema_Autores1_RP_TemasCreadores_AutoresLetra_hl_Creador_0'})]
        letra = [span.get_text(' | ')  for span in page_soup.find_all('span', attrs={'id':'main_Tema1_lbl_Letra'})]
        
        # append the existing dataframe with latest information
        # in order to make arrays of the same length to concatenate dataframe need to add "vacio" for empty strings
        if len(titulo) != 0:
            data.set_value(i, 'titulo', titulo)
        else:
            data.set_value(i, 'titulo', 'vacio')
        
        if len(ritmo) != 0:
            data.set_value(i, 'ritmo', ritmo)
        else:
            data.set_value(i, 'ritmo', 'vacio')

        if len(ano) != 0:
            data.set_value(i, 'ano', ano)
        else:
            data.set_value(i, 'ano', 'vacio')
        
        if len(musica) != 0:
            data.set_value(i, 'musica', musica)
        else:
            data.set_value(i, 'musica', 'vacio')
            
        if len(compositor) != 0:
            data.set_value(i, 'compositor', compositor)
        else:
            data.set_value(i, 'compositor', 'vacio')       
            
        if len(letra) != 0:
            data.set_value(i, 'letra', letra)
        else:
            data.set_value(i, 'letra', 'vacio')
        
        data.to_csv(save_file, encoding='utf-8')
        count += 1
        print count,
        
    return data
        
        #with open(save_file, 'wb') as f:
        #    json.dump(canciones, codecs.getwriter('utf-8')(f), ensure_ascii=False)
    #return canciones

Ejemplo de uso...

In [ ]:
scrap_tango(index_link[0:1000],'tango01.txt')

Read JSON file

In [ ]:

with open('tango01.txt') as data_file:
    t1 = json.load(data_file)

In [ ]:
tangos_final = []
for t in [t1, t2, t3, t4, t5, t6, t7 ,t8, t9, t10]:
    for i in t:
        tangos_final.append(i)

In [ ]:
with open('tango_final.txt', 'wb') as f:
            json.dump(tangos_final, codecs.getwriter('utf-8')(f), ensure_ascii=False)
